In [1]:
#import libraries
import pandas as pd
import os

In [2]:
NFL_Roles=["qb","rb","wr","te","k","dst","dl","lb","db"]
nested_dictionary_frame={}
column_labels={}
for year in range (2018,2024):
    nested_dictionary_frame[str(year)]= {}
    for role in NFL_Roles:
        nested_dictionary_frame[str(year)][role] = {}
        for week in range(1, 19):
            nfl_stats_loop = f"https://www.fantasypros.com/nfl/stats/{role}.php?week={week}&range=week&scoring=PPR&year={year}"

            if role in ["qb", "rb", "wr", "te"]:
                data = pd.read_html(nfl_stats_loop, header=1)
            else:
                data = pd.read_html(nfl_stats_loop, header=0)

            if week == 1:
                column_labels[role] = data[0].columns.tolist()

            nested_dictionary_frame[str(year)][role][str(week)] = data[0]
            nested_dictionary_frame[str(year)][role][str(week)]["Team"]=nested_dictionary_frame[str(year)][role][str(week)]["Player"].str.extract(r'\((\w+)\)')
            nested_dictionary_frame[str(year)][role][str(week)]["Player"]=nested_dictionary_frame[str(year)][role][str(week)]["Player"].replace(r' \(\w+\)', '', regex=True)
            nested_dictionary_frame[str(year)][role][str(week)]["Week"]= week
            nested_dictionary_frame[str(year)][role][str(week)]["Year"]= year
            nested_dictionary_frame[str(year)][role][str(week)]=pd.DataFrame(nested_dictionary_frame[str(year)][role][str(week)].reset_index(drop=True))

In [4]:
for year in nested_dictionary_frame:
    year_folder = f"{year}_football_data"
    os.makedirs(year_folder, exist_ok=True)
    
    for role in nested_dictionary_frame[year]:
        role_folder = os.path.join(year_folder, f"{role}_data")
        os.makedirs(role_folder, exist_ok=True)

        for week in nested_dictionary_frame[year][role]:
            df = nested_dictionary_frame[year][role][week]
            table_name = f"{role}_{week}"
            file_path = os.path.join(role_folder, f"{table_name}.csv")
            df.to_csv(file_path, index=False)